In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

## Labelling of train and test data

In [15]:
import os
df_train=pd.read_excel('train_irisdata.xlsx',sheet_name='Train',engine='openpyxl')
df_test=pd.read_excel('test_irisdata.xlsx',engine='openpyxl')

In [16]:
df_test.shape


(3492, 2)

In [17]:
df_train.shape

(13806, 2)

In [18]:
import re
# Dictionary of english Contractions - move 
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not","can't": "can not","can't've": "cannot have",
"'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
"didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
"hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
"he'll've": "he will have","how'd": "how did","how'd'y": "how do you","how'll": "how will","i'd": "i would",
"i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have",
"isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have",
"let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
"mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
"needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
"shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will",
"she'll've": "she will have","should've": "should have","shouldn't": "should not",
"shouldn't've": "should not have","so've": "so have","that'd": "that would","that'd've": "that would have",
"there'd": "there would","there'd've": "there would have",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have",
"they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would",
"we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
"weren't": "were not","what'll": "what will","what'll've": "what will have","what're": "what are",
"what've": "what have","when've": "when have","where'd": "where did",
"where've": "where have","who'll": "who will","who'll've": "who will have","who've": "who have",
"why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
"would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all",
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
"you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have",
"you're": "you are","you've": "you have"}
# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)
# Expanding Contractions
df_train['utterance']=df_train['utterance'].apply(lambda x:expand_contractions(x))

## Cleaning for train data

1. Current one arjun excluded above 30 ones
2. We should omit utternce>30, or if you want, truncate it and take the latter portion to preserve the valueable information

In [19]:
import re
import string
def clean_text(text):
    text=str(text)
    text=text.lower()
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^A-Za-z0-9]',' ',text)
    return text

def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct

def tokenize(text):
    split=re.split("\W+",text)
    return split

import nltk
from nltk.stem import WordNetLemmatizer
lemm=WordNetLemmatizer()
def remove_stopwords(text):
    stopwords=nltk.corpus.stopwords.words('english')
    text=[lemm.lemmatize(word) for word in tokenize(text) if word not in stopwords]
    return ' '.join(text)

df_train['utterance']=df_train['utterance'].apply(lambda x: clean_text(x))
df_train['utterance']=df_train['utterance'].apply(lambda x: re.sub(' +',' ',x))
df_train['utterance']=df_train['utterance'].str.replace(".", "")
df_train['utterance']=df_train['utterance'].apply(lambda x: remove_punctuation(x))
df_train['utterance']=df_train['utterance'].apply(lambda x: remove_stopwords(x))

df_train['utterance']=df_train['utterance'].str.split().str[-30:].str.join(' ')
df_train['intent']=df_train['intent'].astype(str).str.lower()
# selecting on > 2 len sentence utterances
df_trainutt=df_train["utterance"].values
df_trainutt
s = pd.Series(df_trainutt)
df_train["utt"]=s.str.split().to_frame()
type(s.str.split().to_frame())
df_train['Length'] = df_train['utt'].str.len()
df_train
#last step
df_train.query('Length > 2', inplace = True) 
df_train=df_train[["utterance","intent"]]
df_train

,utterance,intent
0,apply 1bank account password reset,access_management.reset_1bank_password
1,able see application drop box uams,access_management.missing_application_in_uams
2,bulk request available,access_management.process_bulk_request_bulk_ap...
4,1bank id password reset uams,access_management.reset_1bank_password
5,1bank account password reset,access_management.reset_1bank_password
...,...,...
13799,perform penetration test,cybersecurity.penetration_test
13800,perform screen recording,computer.how_to_record_screen
13803,playback recording nice portal,voice_services.playback_recording_from_niceportal
13804,position taskbar desktop,computer.misplaced_taskbar


In [8]:
# import spacy
# nlp = spacy.load('en_core_web_sm',disable=['ner','parser'])
# nlp.max_length=5000000
# # nlp = spacy.load('C:/Users/thiyaneswaranm/Documents/New folder/model_api_gold/en_core_web_sm-2.2.5/en_core_web_sm/en_core_web_sm-2.2.5',disable=['ner','parser'])
# def lemmatize_sent(text):
#     doc = nlp(text)
#     lemm_word=[]
#     for token in doc:
#         if (token.lemma_.casefold())!=("-pron-"):
#     #         print(token, token.lemma_,type(token.lemma_))
#             lemm_word.append(token.lemma_)
#         else:
#             lemm_word.append(token)
#     listToStr = ' '.join([str(elem) for elem in lemm_word]) 
#     return str(listToStr.lower())
# print(lemmatize_sent("my am able min to"))
# df_train['utterance']=df_train['utterance'].apply(lemmatize_sent)

## Cleaning for test data

In [20]:
import re
import string
def clean_text(text):
    text=str(text)
    
    
    text=text.lower()
    #text=re.sub('\n',' ',text)
    #text=re.sub(r"http\S+", "", text)
    text=re.sub('[^A-Za-z0-9]',' ',text)
    return text

def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct

def tokenize(text):
    split=re.split("\W+",text)
    return split

import nltk
from nltk.stem import WordNetLemmatizer
lemm=WordNetLemmatizer()
def remove_stopwords(text):
    stopwords=nltk.corpus.stopwords.words('english')
    text=[lemm.lemmatize(word) for word in tokenize(text) if word not in stopwords]
    return ' '.join(text)

df_test['utterance']=df_test['utterance'].apply(lambda x: clean_text(x))
df_test['utterance']=df_test['utterance'].apply(lambda x: re.sub(' +',' ',x))
df_test['utterance']=df_test['utterance'].str.replace(".", "")
df_test['utterance']=df_test['utterance'].apply(lambda x: remove_punctuation(x))
df_test['utterance']=df_test['utterance'].apply(lambda x: remove_stopwords(x))
df_test['intent']=df_test['intent'].astype(str).str.lower()

df_test['utterance']=df_test['utterance'].str.split().str[-30:].str.join(' ')

In [11]:
df_test

,utterance,intent
0,print f3w,printer_scanner.tap_and_print_frequently_asked...
1,oracle insallation,microsoft_office_365.onedrive_syncing_issue
2,want know retrigger long running oracle query,network.unable_to_access_the_internet
3,access huawei say available desktop please con...,access_management.unable_to_change_password
4,raise dcon,general_enquiry.dev_connect
...,...,...
10539,download center,computer.imac_software_installation
10540,download centre,computer.imac_software_installation
10541,get access key,mobility_suite.blackberry_work_access_key_gene...
10542,check asset id,computer.fixed_asset_list


In [ ]:
# selecting on > 2 len sentence utterances
# df_testutt=df_test["utterance"].values
# df_testutt
# s = pd.Series(df_testutt)
# df_test["utt"]=s.str.split().to_frame()
# type(s.str.split().to_frame())
# df_test['Length'] = df_test['utt'].str.len()
# df_test
# #last step
# df_test.query('Length > 2', inplace = True) 
# df_test=df_test[["utterance","intent"]]
# df_test['utterance'][df_test['utterance']=='']='NONE'
# df_test

1. remove the c intent (less than 15 utterance)(due to class imbalance). when you find c intent, you dont call the prediction and ignore it and mark it as NA

In [ ]:
# more_than_15=df_train['intent'].value_counts()[df_train['intent'].value_counts()>15].index
# df_test['intent'][df_test['intent'].isin(more_than_15)]='NA'

In [ ]:
df_test

## Labelling

In [21]:
#df=pd.concat([df_train[['utterance','intent']],df_test[['utterance','intent']]])
le=preprocessing.LabelEncoder()
le.fit(df_train['intent'])

df_train['intent1']=le.transform(df_train['intent'])
mapping=df_train[['intent','intent1']].drop_duplicates().rename(columns={'intent1':'mapping'})

df_test=pd.merge(df_test,mapping,on='intent',how='left').rename(columns={'mapping':'intent1'})

df_test['intent1']=df_test['intent1'].fillna(0).astype(int)

## Save the files

In [22]:
df_train.to_excel('train_iris_fullcleaned.xlsx',index=None)

writer=pd.ExcelWriter('test_iris_fullcleaned.xlsx',engine='xlsxwriter')

df_test.to_excel(writer,sheet_name='Test',index=None)
mapping.to_excel(writer,sheet_name='Mapping',index=None)

writer.save()